A partire da avalanche 0.4.0 con l'auito del notebook "prova_OCL" tentiamo di costruire un benchmark con esperienze correttamente inserite.

In [1]:
from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.scenarios import OnlineCLScenario
from avalanche.models import SimpleCNN, SimpleMLP
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import accuracy_metrics,\
    forward_transfer_metrics,bwt_metrics,\
    ram_usage_metrics, timing_metrics, EpochAccuracy
from torch.nn import CrossEntropyLoss
from avalanche.training.plugins import EarlyStoppingPlugin
from torch.optim import SGD
import torch
# Install necessary packages (uncomment if running in a new environment)
# !pip install avalanche-lib==0.4.0 torch==2.1.2 matplotlib

import torch
import torch.nn as nn
import torch.optim as optim
from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.supervised import Naive
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, timing_metrics, forgetting_metrics, cpu_usage_metrics
from avalanche.logging import InteractiveLogger
from avalanche.training.plugins import EarlyStoppingPlugin
from avalanche.benchmarks.scenarios import OnlineCLScenario
import matplotlib.pyplot as plt

# Import the SlimResNet18 from the provided file
from slim_resnet18 import SlimResNet18

# Create model function
def create_model():
    return SlimResNet18(nclasses=10, input_size=(3, 32, 32))

# Optimizer
def build_optimizer(model):
    return optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Loss function
def build_criterion():
    return nn.CrossEntropyLoss()

# Evaluation plugin
def build_eval_plugin():
    return EvaluationPlugin(
        accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
        loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
        timing_metrics(epoch=True, stream=True),
        forgetting_metrics(experience=True, stream=True),
        cpu_usage_metrics(experience=True, stream=True),
        loggers=[InteractiveLogger()]
    )


/opt/anaconda3/envs/SDAenv3/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Benchmark ###

ci occupiamo ora dell'inizializzazione del benchmark a partire da CLEAR10.
Colleghiamo la cartella drive dove conserviamo il file pickle e numpy delle nostre immagini.

In [2]:
import numpy as np
import pickle

# Carica da file .npy
#CLEAR10_train = np.load('/content/drive/Othercomputers/Il mio MacBook Pro/cartella drive/CLEAR10_train.npy', allow_pickle=True).item()

# Carica da file .pickle
with open('CLEAR10_train.pickle', 'rb') as handle:
   CLEAR10_train = pickle.load(handle)

Importiamo le librerie per inizializzare il benchmark. A questo scopo sono necessari tensori torch.

In [3]:
import torch
from torch.utils.data.dataset import TensorDataset
from avalanche.benchmarks.utils import make_classification_dataset
from avalanche.benchmarks.generators import dataset_benchmark

In [4]:
CLEAR10_train = CLEAR10_train
tensor_data = torch.tensor(CLEAR10_train, dtype = torch.float)

Inizializziamo allora due vettori di train e test per conservare i dati (non credo sia strettamente necessario salvarli, da valutare)

In [5]:
train_experiences = []
test_experiences = []
torch_datas = []

andiamo allora ad assemblare le componenti per il benchmark.

In [6]:
for i in range(10):
  x_data = tensor_data[i*3300:(i+1)*3300]
  task_dim = x_data.size()[0] # necessario per compatibilità di dimensioni
  y_data = torch.full((task_dim,), i) # task labels
  torch_data = TensorDataset(x_data, y_data)
  torch_datas.append(torch_data)
  dataset = make_classification_dataset(torch_data, task_labels=[i for _ in range(task_dim)])
  train_experiences.append(dataset)

  x_data_test = tensor_data[i*3300:(i+1)*3300]
  y_data_test = torch.full((task_dim,), i) # task labels
  torch_data_test = TensorDataset(x_data_test, y_data_test)
  dataset_test = make_classification_dataset(torch_data_test, task_labels=[i for _ in range(task_dim)])
  test_experiences.append(dataset_test)

passiamo alla vera e propria creazione del benchmark


In [7]:
benchmark = dataset_benchmark(train_datasets = train_experiences, test_datasets = test_experiences)

### Modello ###

importiamo ora il modello Naive

Creiamo il base model, l'evaluation plugin (necessario a definire le metriche da considerare), l'optimizer ed infine una funzione per la loss function

In [8]:
# Create the model
model = create_model()

# Define the strategy
cl_strategy = Naive(
    model,
    build_optimizer(model),
    build_criterion(),
    train_mb_size=10,
    train_epochs=1,
    eval_mb_size=10,
    evaluator=build_eval_plugin(),
    eval_every=1,
)

res = []
minibatches = [] # salvo i minibatches per vedere
for experience in benchmark.train_stream:
  res.append([])
  print("Start of experience ", experience.current_experience)
  print("Current Classes ", experience.classes_in_this_experience)
  ocl_scenario = OnlineCLScenario(
      original_streams=benchmark.streams.values(),
      experiences=experience,
      experience_size=10,
      access_task_boundaries=True,
      shuffle=False
  )

  minibatches.append([])
  for i, minibatch in enumerate(ocl_scenario.train_stream):
    print("Minibatch: ", i+1)
    print(minibatch)
    res[-1].append(cl_strategy.train(minibatch, eval_streams=[benchmark.test_stream]))
    minibatches[-1].append(minibatch)
  print()

Start of experience  0
Current Classes  [0]
Minibatch:  1
-- >> Start of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 330/330 [00:22<00:00, 14.57it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task000/Exp000 = 105.1656
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 6.6482
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0000
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 330/330 [00:14<00:00, 22.20it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	CPUUsage_Exp/eval_phase/test_stream/Task001/Exp001 = 147.7725
	Loss_Exp/eval_phase/test_stream/Task001/Exp001 = 1.8062
	Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001 = 0.0003
-- Starting eval on experience 2 (Task 2) from test stream --
100%|██████████| 330/330 [00:17<00:00, 18.91it/s]
> Eval on experience 2 (Task 2) from test stream ended.
	CPUUsag